In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/maternal-and-child-health-monitoring-in-lmics/training_label.csv
/kaggle/input/maternal-and-child-health-monitoring-in-lmics/sample submission.csv
/kaggle/input/maternal-and-child-health-monitoring-in-lmics/gee_features.csv
/kaggle/input/selecting-features-using-tree/y_train.csv
/kaggle/input/selecting-features-using-tree/x_train.csv
/kaggle/input/selecting-features-using-tree/__results__.html
/kaggle/input/selecting-features-using-tree/x_submission.csv
/kaggle/input/selecting-features-using-tree/__notebook__.ipynb
/kaggle/input/selecting-features-using-tree/DHSID.csv
/kaggle/input/selecting-features-using-tree/__output__.json
/kaggle/input/selecting-features-using-tree/custom.css


Loading the required libraries and packages

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import tqdm
from scipy import stats
from torch.utils.tensorboard import SummaryWriter

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

Checking whether cuda is available or not

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE IS ... ", device)

DEVICE IS ...  cpu


In [101]:
epochs = 5
lr = 1e-4
batch_size = 32

Loading the datsets

In [6]:
x_train = pd.read_csv('/kaggle/input/selecting-features-using-tree/x_train.csv')
y_train = pd.read_csv('/kaggle/input/selecting-features-using-tree/y_train.csv')
x_submission = pd.read_csv('/kaggle/input/selecting-features-using-tree/x_submission.csv')
DHSID = pd.read_csv('/kaggle/input/selecting-features-using-tree/DHSID.csv')

Scaling the data

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Stack the DataFrames vertically
merged_df = pd.concat([x_train, x_submission], axis=0)

# Separate the column names from the DataFrame
column_names = merged_df.columns.tolist()

scaled_data = scaler.fit_transform(merged_df)

scaled_df = pd.DataFrame(scaled_data, columns=column_names)

In [8]:
# Separating the DataFrame back into the original parts
x_train_scaled = scaled_df[:len(x_train)]
x_submission_scaled = scaled_df[len(x_train):]

Splitting the data

In [9]:
from sklearn.model_selection import train_test_split

# Set the random seed for reproducibility
random_seed = 42
np.random.seed(random_seed)

# Splitting the data into train and test sets based on country (80% train, 20% test)
x_train1, x_test1, y_train1, y_test1 = train_test_split(x_train_scaled, y_train, test_size=0.2, stratify = x_train_scaled["key1"])

# Splitting the train set into train and val sets based on country (75% train, 25% dev)
x_train1, x_val1, y_train1, y_val1 = train_test_split(x_train1, y_train1, test_size=0.25, stratify = x_train1["key1"])

In [10]:
y_train1.shape

(46768, 6)

In [11]:
x_test1.shape

(15590, 2000)

In [12]:
x_val1.shape

(15590, 2000)

In [13]:
y_train.columns

Index(['Mean_BMI', 'Median_BMI', 'Unmet_Need_Rate', 'Under5_Mortality_Rate',
       'Skilled_Birth_Attendant_Rate', 'Stunted_Rate'],
      dtype='object')

**Multitask learning model**

In [68]:
class MultiTaskLearn(nn.Module):
    def __init__(self, embed_dim=2000, layers=[2000, 512, 256, 256, 128]):
        super().__init__()

        self.embedding_dim = embed_dim

        self.mlp_network = nn.Sequential(
            nn.Linear(layers[0], layers[1]),  
            nn.ReLU(),
            nn.Linear(layers[1], layers[2]), 
            nn.ReLU(),
            nn.Linear(layers[2], layers[3]),
            nn.ReLU(),
        )

        self.last_layer_mean_bmi = nn.Linear(
            layers[3], 1
        )  
        self.last_layer_median_bmi = nn.Linear(
            layers[3], 1
        )  
        self.last_layer_Unmet_Need_Rate = nn.Linear(
            layers[3], 1
        )  
        self.last_layer_Under5_Mortality_Rate = nn.Linear(
            layers[3], 1
        )  
        self.last_layer_Skilled_Birth_Attendant_Rate = nn.Linear(
            layers[3], 1
        )  
        self.last_layer_Stunted_Rate = nn.Linear(
            layers[3], 1
        ) 

    def forward(self, x):
        x = self.mlp_network(x)
 
        out_1 = self.last_layer_mean_bmi(x)
        out_2 = self.last_layer_median_bmi(x)
        out_3 = self.last_layer_Unmet_Need_Rate(x)
        out_4 = self.last_layer_Under5_Mortality_Rate(x)
        out_5 = self.last_layer_Skilled_Birth_Attendant_Rate(x)
        out_6 = self.last_layer_Stunted_Rate(x)

        return out_1, out_2, out_3, out_4, out_5, out_6 

In [15]:
y_train.columns

Index(['Mean_BMI', 'Median_BMI', 'Unmet_Need_Rate', 'Under5_Mortality_Rate',
       'Skilled_Birth_Attendant_Rate', 'Stunted_Rate'],
      dtype='object')

**Collate function** : process data samples before they are passed to the data loader

In [16]:
def collate_fn(data):
    x, y = data
    x_inp = torch.tensor(x.values, dtype=torch.float32, device=device)
    y_out1 = torch.tensor(y["Mean_BMI"].values, dtype=torch.float32, device=device)
    y_out2 = torch.tensor(y["Median_BMI"].values, dtype=torch.float32, device=device)
    y_out3 = torch.tensor(y["Unmet_Need_Rate"].values, dtype=torch.float32, device=device)
    y_out4 = torch.tensor(y["Under5_Mortality_Rate"].values, dtype=torch.float32, device=device)
    y_out5 = torch.tensor(y["Skilled_Birth_Attendant_Rate"].values, dtype=torch.float32, device=device)
    y_out6 = torch.tensor(y["Stunted_Rate"].values, dtype=torch.float32, device=device)
    return x_inp, y_out1, y_out2, y_out3, y_out4, y_out5, y_out6

Dataloaders

In [17]:
train_dataloader = DataLoader(
    TensorDataset(*collate_fn((x_train1, y_train1))), batch_size=batch_size
)

In [18]:
test_dataloader = DataLoader(
    TensorDataset(*collate_fn((x_test1, y_test1))), batch_size=batch_size
)

In [19]:
val_dataloader = DataLoader(
    TensorDataset(*collate_fn((x_val1, y_val1))), batch_size=batch_size
)

**Model calling**

In [75]:
model = MultiTaskLearn(embed_dim=2000, layers=[2000, 1000, 500, 100]).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

Model evaluation function for selecting the best layers and sizes

In [71]:
def model_evaluation(model, dataloader):
    mse_loss = []
    
    with torch.no_grad():
        for i, batch in enumerate(dataloader):
            x, y1, y2, y3, y4, y5, y6 = batch
            out_1, out_2, out_3, out_4, out_5, out_6 = model(x)
            out_1, out_2, out_3, out_4, out_5, out_6 = out_1.squeeze(), out_2.squeeze(), out_3.squeeze(), out_4.squeeze(), out_5.squeeze(), out_6.squeeze()
            loss_1 = criterion(out_1, y1)
            loss_2 = criterion(out_2, y2)
            loss_3 = criterion(out_3, y3)
            loss_4 = criterion(out_4, y4)
            loss_5 = criterion(out_5, y5)
            loss_6 = criterion(out_6, y6)
            loss = loss_1 + loss_2 + loss_3 + loss_4 + loss_5 + loss_6
            mse_loss.append(loss.item())

    mse_loss_avg = np.array(mse_loss).mean()
    return mse_loss_avg

Selection of **best number of layers and sizes of layers**

In [72]:
best_model = None
best_loss = float('inf')
best_layers = None

# Define a set of hyperparameters to search over
layer_sizes_list = [[1000, 500, 100], [1500, 750, 250], [2000, 1000, 500]]
num_layers_list = [3, 4, 5]

for layer_sizes in layer_sizes_list:
    for num_layers in num_layers_list:
        # Create and train the model with the current hyperparameters
        model = MultiTaskLearn(embed_dim=2000, layers=[2000] + layer_sizes * (num_layers - 1)).to(device)
        criterion = nn.MSELoss()
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
        for epoch in range(epochs):
            print("Training of the Multi Task Learning model ")
            training_loss = []
            mse_1 = []
            mse_2 = []
            mse_3 = []
            mse_4 = []
            mse_5 = []
            mse_6 = []

            i = 0
            for batch in train_dataloader:
                x, y1, y2, y3, y4, y5, y6 = batch
                out_1, out_2, out_3, out_4, out_5, out_6 = model(x)
                out_1, out_2, out_3, out_4, out_5, out_6 = out_1.squeeze(), out_2.squeeze(), out_3.squeeze(),out_4.squeeze(),out_5.squeeze(),out_6.squeeze()

                loss_1 = criterion(out_1, y1)
                loss_2 = criterion(out_2, y2)
                loss_3 = criterion(out_3, y3)
                loss_4 = criterion(out_4, y4)
                loss_5 = criterion(out_5, y5)
                loss_6 = criterion(out_6, y6)
                loss = loss_1 + loss_2 + loss_3 + loss_4 + loss_5 + loss_6

                training_loss.append(loss.item())
                mse_1.append(loss_1.item())
                mse_2.append(loss_2.item())
                mse_3.append(loss_3.item())
                mse_4.append(loss_4.item())
                mse_5.append(loss_5.item())
                mse_6.append(loss_6.item())

                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                i += 1
    
            training_loss_avg = np.array(training_loss).mean()
            train_mse_1 = np.array(mse_1).mean()
            train_mse_2 = np.array(mse_2).mean()
            train_mse_3 = np.array(mse_3).mean()
            train_mse_4 = np.array(mse_4).mean()
            train_mse_5 = np.array(mse_5).mean()
            train_mse_6 = np.array(mse_6).mean()
            
            print('For validation data...')
            model_evaluation(model,val_dataloader)
            

        # Evaluate the model on the validation set
        val_loss = model_evaluation(model, val_dataloader)

        # Check if this is the best model so far
        if val_loss < best_loss:
            best_model = model
            best_loss = val_loss
            best_layers = (num_layers, layer_sizes)

print("Best number of layers and layer sizes:", best_layers)
print("Best validation loss:", best_loss)

# Finally, evaluate the best model on the test set
print("Final evaluation on the test set:")
model_evaluation(best_model, test_dataloader)

Training of the Multi Task Learning model 
For validation data...
Training of the Multi Task Learning model 
For validation data...
Training of the Multi Task Learning model 
For validation data...
Training of the Multi Task Learning model 
For validation data...
Training of the Multi Task Learning model 
For validation data...
Training of the Multi Task Learning model 
For validation data...
Training of the Multi Task Learning model 
For validation data...
Training of the Multi Task Learning model 
For validation data...
Training of the Multi Task Learning model 
For validation data...
Training of the Multi Task Learning model 
For validation data...
Training of the Multi Task Learning model 
For validation data...
Training of the Multi Task Learning model 
For validation data...
Training of the Multi Task Learning model 
For validation data...
Training of the Multi Task Learning model 
For validation data...
Training of the Multi Task Learning model 
For validation data...
Training o

1073.2310667194304

So from this, we are selecting the number of layers as **3** and the size of each layers is **1000,500,100** respectively

Model evaluation function for showing the **squared pearson r** value of the test and validation data

In [86]:
def model_evaluation(model, dataloader):
    mse_1 = []
    mse_2 = []
    mse_3 = []
    mse_4 = []
    mse_5 = []
    mse_6 = []

    preds_1 = []
    preds_2 = []
    preds_3 = []
    preds_4 = []
    preds_5 = []
    preds_6 = []

    y_1 = []
    y_2 = []
    y_3 = []
    y_4 = []
    y_5 = []
    y_6 = []

    mse_loss = []

    for i, batch in enumerate(dataloader):
        x, y1, y2, y3, y4, y5, y6 = batch
        with torch.no_grad():
            out_1, out_2, out_3, out_4, out_5, out_6 = model(x)
            out_1, out_2, out_3, out_4, out_5, out_6 = out_1.squeeze(), out_2.squeeze(), out_3.squeeze(), out_4.squeeze(), out_5.squeeze(), out_6.squeeze()
            loss_1 = criterion(out_1, y1)
            loss_2 = criterion(out_2, y2)
            loss_3 = criterion(out_3, y3)
            loss_4 = criterion(out_4, y4)
            loss_5 = criterion(out_5, y5)
            loss_6 = criterion(out_6, y6)
            loss = loss_1 + loss_2 + loss_3 + loss_4 + loss_5 + loss_6

            mse_1.append(loss_1.item())
            mse_2.append(loss_2.item())
            mse_3.append(loss_3.item())
            mse_4.append(loss_4.item())
            mse_5.append(loss_5.item())
            mse_6.append(loss_6.item())
            mse_loss.append(loss.item())

            y_1.append(y1.cpu().numpy())
            y_2.append(y2.cpu().numpy())
            y_3.append(y3.cpu().numpy())
            y_4.append(y4.cpu().numpy())
            y_5.append(y5.cpu().numpy())
            y_6.append(y6.cpu().numpy())

            preds_1_numpy = out_1.detach().cpu().numpy()
            preds_1.append(preds_1_numpy)
            preds_2_numpy = out_2.detach().cpu().numpy()
            preds_2.append(preds_2_numpy)
            preds_3_numpy = out_3.detach().cpu().numpy()
            preds_3.append(preds_3_numpy)
            preds_4_numpy = out_4.detach().cpu().numpy()
            preds_4.append(preds_4_numpy)
            preds_5_numpy = out_5.detach().cpu().numpy()
            preds_5.append(preds_5_numpy)
            preds_6_numpy = out_6.detach().cpu().numpy()
            preds_6.append(preds_6_numpy)

    # After the for loop, converting lists to tensors
    y_1 = torch.tensor(np.concatenate(y_1, axis=0))
    y_2 = torch.tensor(np.concatenate(y_2, axis=0))
    y_3 = torch.tensor(np.concatenate(y_3, axis=0))
    y_4 = torch.tensor(np.concatenate(y_4, axis=0))
    y_5 = torch.tensor(np.concatenate(y_5, axis=0))
    y_6 = torch.tensor(np.concatenate(y_6, axis=0))

    # Converting predictions lists to arrays
    preds_1 = np.concatenate(preds_1, axis=0)
    preds_2 = np.concatenate(preds_2, axis=0)
    preds_3 = np.concatenate(preds_3, axis=0)
    preds_4 = np.concatenate(preds_4, axis=0)
    preds_5 = np.concatenate(preds_5, axis=0)
    preds_6 = np.concatenate(preds_6, axis=0)

    mse_loss_avg = np.array(mse_loss).mean()
    mse_1_avg = np.array(mse_1).mean()
    mse_2_avg = np.array(mse_2).mean()
    mse_3_avg = np.array(mse_3).mean()
    mse_4_avg = np.array(mse_4).mean()
    mse_5_avg = np.array(mse_5).mean()
    mse_6_avg = np.array(mse_6).mean()
    print('average loss is {}'.format(mse_loss_avg))

    r2_column1, _ = stats.pearsonr(preds_1, y_1)
    r2_column1 = r2_column1**2
    print('r2 of column1 is {}'.format(r2_column1))
    r2_column2, _ = stats.pearsonr(preds_2, y_2)
    r2_column2 = r2_column2**2
    print('r2 of column2 is {}'.format(r2_column2))
    r2_column3, _ = stats.pearsonr(preds_3, y_3)
    r2_column3 = r2_column3**2
    print('r2 of column3 is {}'.format(r2_column3))
    r2_column4, _ = stats.pearsonr(preds_4, y_4)
    r2_column4 = r2_column4**2
    print('r2 of column4 is {}'.format(r2_column4))
    r2_column5, _ = stats.pearsonr(preds_5, y_5)
    r2_column5 = r2_column5**2
    print('r2 of column5 is {}'.format(r2_column5))
    r2_column6, _ = stats.pearsonr(preds_6, y_6)
    r2_column6 = r2_column6**2
    print('r2 of column6 is {}'.format(r2_column6))

Training the model

In [76]:
for epoch in range(epochs):
    print("Training of the Multi Task Learning model ")
    training_loss = []
    mse_1 = []
    mse_2 = []
    mse_3 = []
    mse_4 = []
    mse_5 = []
    mse_6 = []

    i = 0
    for batch in train_dataloader:
        x, y1, y2, y3, y4, y5, y6 = batch
        out_1, out_2, out_3, out_4, out_5, out_6 = model(x)
        out_1, out_2, out_3, out_4, out_5, out_6 = out_1.squeeze(), out_2.squeeze(), out_3.squeeze(),out_4.squeeze(),out_5.squeeze(),out_6.squeeze()

        loss_1 = criterion(out_1, y1)
        loss_2 = criterion(out_2, y2)
        loss_3 = criterion(out_3, y3)
        loss_4 = criterion(out_4, y4)
        loss_5 = criterion(out_5, y5)
        loss_6 = criterion(out_6, y6)
        loss = loss_1 + loss_2 + loss_3 + loss_4 + loss_5 + loss_6

        training_loss.append(loss.item())
        mse_1.append(loss_1.item())
        mse_2.append(loss_2.item())
        mse_3.append(loss_3.item())
        mse_4.append(loss_4.item())
        mse_5.append(loss_5.item())
        mse_6.append(loss_6.item())

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        i += 1
    
    training_loss_avg = np.array(training_loss).mean()
    train_mse_1 = np.array(mse_1).mean()
    train_mse_2 = np.array(mse_2).mean()
    train_mse_3 = np.array(mse_3).mean()
    train_mse_4 = np.array(mse_4).mean()
    train_mse_5 = np.array(mse_5).mean()
    train_mse_6 = np.array(mse_6).mean()
    print('For test data....')
    model_evaluation(model,test_dataloader)
    print('For validation data...')
    model_evaluation(model,val_dataloader)

Training of the Multi Task Learning model 
For test data....
average loss is 1202.1843070358527
r2 of column1 is 0.30930042622374504
r2 of column2 is 0.32080338330315705
r2 of column3 is 0.49522479782894346
r2 of column4 is 0.2986705413758222
r2 of column5 is 0.4371420416765243
r2 of column6 is 0.28268565423212955
For validation data...
average loss is 1174.1937646709505
r2 of column1 is 0.3123601339983473
r2 of column2 is 0.32576116507950015
r2 of column3 is 0.5109982397545553
r2 of column4 is 0.3058558219076788
r2 of column5 is 0.44367896739196205
r2 of column6 is 0.2991372388478716
Training of the Multi Task Learning model 
For test data....
average loss is 1128.0763547303247
r2 of column1 is 0.3955947440382598
r2 of column2 is 0.39777350977233583
r2 of column3 is 0.5194993421856449
r2 of column4 is 0.334116629022681
r2 of column5 is 0.4785398033790418
r2 of column6 is 0.31387460278426427
For validation data...
average loss is 1102.463246329886
r2 of column1 is 0.3994911664968064
r2

Now evaluating for **submission data**

In [77]:
train_data_full_dataloader = DataLoader(
    TensorDataset(*collate_fn((x_train_scaled, y_train))), batch_size=batch_size
)

In [102]:
for epoch in range(epochs):
    print("Training of the Multi Task Learning model for prediction")
    for batch in train_data_full_dataloader:
        x, y1, y2, y3, y4, y5, y6 = batch
        out_1, out_2, out_3, out_4, out_5, out_6 = model(x)
        out_1, out_2, out_3, out_4, out_5, out_6 = out_1.squeeze(), out_2.squeeze(), out_3.squeeze(),out_4.squeeze(),out_5.squeeze(),out_6.squeeze()

Training of the Multi Task Learning model for prediction
Training of the Multi Task Learning model for prediction
Training of the Multi Task Learning model for prediction
Training of the Multi Task Learning model for prediction
Training of the Multi Task Learning model for prediction


In [79]:
x_submission_tensor = torch.tensor(x_submission_scaled.values, dtype=torch.float32, device=device)

In [80]:
model.eval()

MultiTaskLearn(
  (mlp_network): Sequential(
    (0): Linear(in_features=2000, out_features=1000, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1000, out_features=500, bias=True)
    (3): ReLU()
    (4): Linear(in_features=500, out_features=100, bias=True)
    (5): ReLU()
  )
  (last_layer_mean_bmi): Linear(in_features=100, out_features=1, bias=True)
  (last_layer_median_bmi): Linear(in_features=100, out_features=1, bias=True)
  (last_layer_Unmet_Need_Rate): Linear(in_features=100, out_features=1, bias=True)
  (last_layer_Under5_Mortality_Rate): Linear(in_features=100, out_features=1, bias=True)
  (last_layer_Skilled_Birth_Attendant_Rate): Linear(in_features=100, out_features=1, bias=True)
  (last_layer_Stunted_Rate): Linear(in_features=100, out_features=1, bias=True)
)

In [103]:
# Make predictions
with torch.no_grad():
    out_1, out_2, out_3, out_4, out_5, out_6 = model(x_submission_tensor)
    out_1, out_2, out_3, out_4, out_5, out_6 = (
        out_1.squeeze().cpu().numpy(),
        out_2.squeeze().cpu().numpy(),
        out_3.squeeze().cpu().numpy(),
        out_4.squeeze().cpu().numpy(),
        out_5.squeeze().cpu().numpy(),
        out_6.squeeze().cpu().numpy(),
    )

In [104]:
submission_df = pd.DataFrame(
    {
        "Mean_BMI": out_1,
        "Median_BMI": out_2,
        "Unmet_Need_Rate": out_3,
        "Under5_Mortality_Rate": out_4,
        "Skilled_Birth_Attendant_Rate": out_5,
        "Stunted_Rate": out_6,
    }
)

In [105]:
submission_df

,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
0,26.243416,25.924145,40.778206,3.112854,72.814270,12.554750
1,26.242855,25.935745,40.969955,3.078196,71.280876,12.051136
2,24.683704,24.363781,41.692085,3.629073,65.287247,13.288667
3,25.073139,24.751678,32.125931,1.528121,74.187904,9.784106
4,24.474855,24.235176,36.990551,2.135711,70.717033,11.025850
...,...,...,...,...,...,...
14995,23.646233,22.745113,23.654175,7.972811,73.180389,29.109512
14996,26.154629,25.739574,20.133526,6.626734,86.554520,28.000225
14997,24.068684,23.344154,17.618486,4.996466,84.460472,19.504204
14998,25.346584,24.919716,16.007450,5.925478,92.712708,24.596518


In [106]:
# Concatenate the DataFrames horizontally
submission7 = pd.concat([DHSID, submission_df], axis=1)

Submission file

In [107]:
submission7.to_csv('submission7.csv',index = False)